In [1]:
import pandas as pd
import numpy as np
import datetime
from pandasql import sqldf

In [3]:
# Read season data and create a list of all seasons

s162 = pd.read_csv("Data/WCD2016-2.csv")
s163 = pd.read_csv("Data/WCD2016-3.csv")
s171 = pd.read_csv("Data/WCD2017-1.csv")
s172 = pd.read_csv("Data/WCD2017-2.csv")
s173 = pd.read_csv("Data/WCD2017-3.csv")
s181 = pd.read_csv("Data/WCD2018-1.csv")
s182 = pd.read_csv("Data/WCD2018-2.csv")
s183 = pd.read_csv("Data/WCD2018-3.csv")
s191 = pd.read_csv("Data/WCD2019-1.csv")
s192 = pd.read_csv("Data/WCD2019-2.csv")
s193 = pd.read_csv("Data/WCD2019-3.csv")
s201 = pd.read_csv("Data/WCD2020-1.csv")

sealist = [s162, s163, s171, s172, s173, s181, s182, s183, s191, s192, s193, s201]

In [4]:
# Make function for all time features

def new_date(season):
    season['Season_Day'] = 0
    season['DOW'] = 0

    season['New_Date'] = pd.to_datetime(season['Date'])
    season['Season_Day'] = season['New_Date'] - min(season['New_Date'])
    season['Season_Day'] = season['Season_Day'].dt.days
    
    for i in range(0, len(season)):
        season['DOW'][i] = season['New_Date'][i].day_name()

    season['Tuesday'] = season['DOW'] == 'Tuesday'

In [5]:
# Apply function to all seasons

for season in sealist:
    new_date(season)

<ipython-input-4-17477c1a1ce3>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season['DOW'][i] = season['New_Date'][i].day_name()
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [6]:
# Make non-time features function

def feats(season):
    season['Home'] = season['Home/Away'] == 'Home'
    season['Cricket'] = season['Game'] == 'Cricket'
    season['301'] = (season['Game'] == '301 DIDO') | (season['Game'] == '301 SIDO')
    season['Win'] = season['W/A'] == 'W'
    season['Assist'] = season['W/A'] == 'A'
    season['Set'] = pd.array(np.floor(season['Leg_Code']), dtype=pd.Int64Dtype())
    season['Leg'] = pd.array(round((season['Leg_Code'] - season['Set'])*10), dtype=pd.Int64Dtype())
    season['Cork/Start'] = season['Turn_Sequence'] == 1
    season['TDA'] = (season['Points/Marks']/season['Darts_Thrown']) * 3

In [7]:
# Apply function to all seasons
for season in sealist:
    feats(season)

In [8]:
# Single dataframe for all seasons

wcd = pd.concat(sealist)

In [9]:
# Break apart TDA for distinct games
wcd['TDAC'] = wcd['TDA'].mask(wcd['Cricket']==False)
wcd['TDA3'] = wcd['TDA'].mask(wcd['301']==False)
wcd['TDA5'] = wcd['TDA'].mask(wcd['Cricket'] | wcd['301'])

In [10]:
wcd.reset_index(inplace=True)

In [11]:
wcd

,index,Unnamed: 0,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,W/A,...,301,Win,Assist,Set,Leg,Cork/Start,TDA,TDAC,TDA3,TDA5
0,0,0,Bacon,6-7-16,Home,1.1,501 SIDO,24.6,Jason Callan,W,...,False,True,False,1,1,True,24.639344,NaN,NaN,24.639344
1,1,1,Motley Crew,6-7-16,Away,1.1,501 SIDO,24.4,Ken Molitoris,0,...,False,False,False,1,1,False,24.350000,NaN,NaN,24.350000
2,2,2,Bacon,6-7-16,Home,1.2,501 SIDO,32.0,Jason Callan,W,...,False,True,False,1,2,False,31.978723,NaN,NaN,31.978723
3,3,3,Motley Crew,6-7-16,Away,1.2,501 SIDO,30.8,Ken Molitoris,0,...,False,False,False,1,2,True,30.812500,NaN,NaN,30.812500
4,4,4,Bacon,6-7-16,Home,2.1,301 DIDO,31.4,Kurt Nishi,A,...,True,False,True,2,1,False,31.400000,NaN,31.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150430,12725,69,Dark Nights,2-18-20,Away,9.2,501 SIDO,70.0,Jacob Zawa,0,...,False,False,False,9,2,False,70.000000,NaN,NaN,70.000000
150431,12726,70,Cork Busters,2-18-20,Home,9.3,501 SIDO,45.5,Tim Gilmore,0,...,False,False,False,9,3,False,45.500000,NaN,NaN,45.500000
150432,12727,71,Cork Busters,2-18-20,Home,9.3,501 SIDO,71.8,Eddie Palacios,0,...,False,False,False,9,3,False,71.750000,NaN,NaN,71.750000
150433,12728,72,Dark Nights,2-18-20,Away,9.3,501 SIDO,70.6,Karl Hamilton,W,...,False,True,False,9,3,True,70.600000,NaN,NaN,70.600000


In [12]:
# Prepare ordinal dates from the New_Date timestamps
orddate = wcd['New_Date'].copy()

In [13]:
for i in range(0,len(orddate)):
    orddate[i] = orddate[i].toordinal()

In [14]:
wcd['Ord_Date'] = orddate

In [15]:
wcd.to_csv('WCDSQL.csv')

In [31]:
#The object now is to convert current TDA's into 3 separate columns per game, so 9 total.
wcd['TDAC1'] = wcd['TDAC'].mask((wcd['Set'] != 1) & (wcd['Set'] != 4) & (wcd['Set'] != 7))
wcd['TDAC2'] = wcd['TDAC'].mask((wcd['Set'] != 2) & (wcd['Set'] != 5) & (wcd['Set'] != 8))
wcd['TDAC3'] = wcd['TDAC'].mask((wcd['Set'] != 3) & (wcd['Set'] != 6) & (wcd['Set'] != 9))


wcd['TDA31'] = wcd['TDA3'].mask((wcd['Set'] != 1) & (wcd['Set'] != 4) & (wcd['Set'] != 7))
wcd['TDA32'] = wcd['TDA3'].mask((wcd['Set'] != 2) & (wcd['Set'] != 5) & (wcd['Set'] != 8))
wcd['TDA33'] = wcd['TDA3'].mask((wcd['Set'] != 3) & (wcd['Set'] != 6) & (wcd['Set'] != 9))


wcd['TDA51'] = wcd['TDA5'].mask((wcd['Set'] != 1) & (wcd['Set'] != 4) & (wcd['Set'] != 7))
wcd['TDA52'] = wcd['TDA5'].mask((wcd['Set'] != 2) & (wcd['Set'] != 5) & (wcd['Set'] != 8))
wcd['TDA53'] = wcd['TDA5'].mask((wcd['Set'] != 3) & (wcd['Set'] != 6) & (wcd['Set'] != 9))

In [33]:
#Same process for Cork/Start, Win, And Assist
wcd['Win1'] = wcd['Win'].mask((wcd['Set'] != 1) & (wcd['Set'] != 4) & (wcd['Set'] != 7))
wcd['Win2'] = wcd['Win'].mask((wcd['Set'] != 2) & (wcd['Set'] != 5) & (wcd['Set'] != 8))
wcd['Win3'] = wcd['Win'].mask((wcd['Set'] != 3) & (wcd['Set'] != 6) & (wcd['Set'] != 9))


wcd['Assist1'] = wcd['Assist'].mask((wcd['Set'] != 1) & (wcd['Set'] != 4) & (wcd['Set'] != 7))
wcd['Assist2'] = wcd['Assist'].mask((wcd['Set'] != 2) & (wcd['Set'] != 5) & (wcd['Set'] != 8))
wcd['Assist3'] = wcd['Assist'].mask((wcd['Set'] != 3) & (wcd['Set'] != 6) & (wcd['Set'] != 9))


wcd['Cork/Start1'] = wcd['Cork/Start'].mask((wcd['Set'] != 1) & (wcd['Set'] != 4) & (wcd['Set'] != 7))
wcd['Cork/Start2'] = wcd['Cork/Start'].mask((wcd['Set'] != 2) & (wcd['Set'] != 5) & (wcd['Set'] != 8))
wcd['Cork/Start3'] = wcd['Cork/Start'].mask((wcd['Set'] != 3) & (wcd['Set'] != 6) & (wcd['Set'] != 9))

In [34]:
wcd.head(35)

,index,Unnamed: 0,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,W/A,...,TDA53,Win1,Win2,Win3,Assist1,Assist2,Assist3,Cork/Start1,Cork/Start2,Cork/Start3
0,0,0,Bacon,6-7-16,Home,1.1,501 SIDO,24.6,Jason Callan,W,...,NaN,1.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN
1,1,1,Motley Crew,6-7-16,Away,1.1,501 SIDO,24.4,Ken Molitoris,0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
2,2,2,Bacon,6-7-16,Home,1.2,501 SIDO,32.0,Jason Callan,W,...,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN
3,3,3,Motley Crew,6-7-16,Away,1.2,501 SIDO,30.8,Ken Molitoris,0,...,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN
4,4,4,Bacon,6-7-16,Home,2.1,301 DIDO,31.4,Kurt Nishi,A,...,NaN,NaN,0.0,NaN,NaN,1.0,NaN,NaN,0.0,NaN
5,5,5,Bacon,6-7-16,Home,2.1,301 DIDO,30.9,Matt Swift,W,...,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
6,6,6,Motley Crew,6-7-16,Away,2.1,301 DIDO,45.6,Kevin Askham,0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,1.0,NaN
7,7,7,Bacon,6-7-16,Home,2.1,301 DIDO,8.4,Unlisted Player 1,0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
8,8,8,Bacon,6-7-16,Home,2.2,Cricket,1.4,Matt Swift,0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN
9,9,9,Bacon,6-7-16,Home,2.2,Cricket,0.9,Kurt Nishi,0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0.0,NaN


# Start of FiGMA Tables and Calculations

In [35]:
# Make hinge function of the same format as the FiGMA table
stuart = wcd.groupby(['Team','Set','Game','New_Date'])['TDAC1','TDAC2','TDAC3','TDA31','TDA32','TDA33','TDA51','TDA52','TDA53','Win1','Win2','Win3','Assist1','Assist2','Assist3','Cork/Start1','Cork/Start2','Cork/Start3'].mean()

<ipython-input-35-d47d684ef78d>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  stuart = wcd.groupby(['Team','Set','Game','New_Date'])['TDAC1','TDAC2','TDAC3','TDA31','TDA32','TDA33','TDA51','TDA52','TDA53','Win1','Win2','Win3','Assist1','Assist2','Assist3','Cork/Start1','Cork/Start2','Cork/Start3'].mean()


In [36]:
figma = stuart.rolling(5, min_periods=1).mean()

In [38]:
#FiGMA shift (moving date forward to next game for each leg)
figma = figma.shift(1)

In [40]:
# We need to add all these into a single column
figma.to_csv("ShiftedFiGMA.csv")

In [46]:
# Add each game's TDA's into a single column
# Convert NaNs to zeros.
figma['TDA51'] = figma['TDA51'].fillna(0)
figma['TDA52'] = figma['TDA52'].fillna(0)
figma['TDA53'] = figma['TDA53'].fillna(0)
figma['TDA31'] = figma['TDA31'].fillna(0)
figma['TDA32'] = figma['TDA32'].fillna(0)
figma['TDA33'] = figma['TDA33'].fillna(0)
figma['TDAC1'] = figma['TDAC1'].fillna(0)
figma['TDAC2'] = figma['TDAC2'].fillna(0)
figma['TDAC3'] = figma['TDAC3'].fillna(0)

figma['TDA5'] = figma['TDA51'] + figma['TDA52'] + figma['TDA53']
figma['TDA3'] = figma['TDA31'] + figma['TDA32'] + figma['TDA33']
figma['TDAC'] = figma['TDAC1'] + figma['TDAC2'] + figma['TDAC3']

In [48]:
# Retrieve Five Game Moving Average for WCD from FiGMA. 

wcd['FiGMA_TDA5'] = 0.0
for i in range(0,len(wcd)):
    wcd['FiGMA_TDA5'][i] = figma['TDA5'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]

wcd['FiGMA_TDA3'] = 0.0
for i in range(0,len(wcd)):
    wcd['FiGMA_TDA3'][i] = figma['TDA3'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]

wcd['FiGMA_TDAC'] = 0.0
for i in range(0,len(wcd)):
    wcd['FiGMA_TDAC'][i] = figma['TDAC'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]


<ipython-input-48-8ba033d1d62c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wcd['FiGMA_TDA5'][i] = figma['TDA5'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]
<ipython-input-48-8ba033d1d62c>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wcd['FiGMA_TDA3'][i] = figma['TDA3'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]
<ipython-input-48-8ba033d1d62c>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-v

In [71]:
#Rock, paper, scissors loops for win%, assist%, start%
for i in range (0,len(figma)):
    if (figma['Win3'].iloc[i] > 0) & (figma['Win1'].iloc[i] > 0):
        figma['Win3'].iloc[i] = 0
    if (figma['Win1'].iloc[i] > 0) & (figma['Win2'].iloc[i] > 0):
        figma['Win1'].iloc[i] = 0
    if (figma['Win2'].iloc[i] > 0) & (figma['Win3'].iloc[i] > 0):
        figma['Win2'].iloc[i] = 0

for i in range (0,len(figma)):
    if (figma['Assist3'].iloc[i] > 0) & (figma['Assist1'].iloc[i] > 0):
        figma['Assist3'].iloc[i] = 0
    if (figma['Assist1'].iloc[i] > 0) & (figma['Assist2'].iloc[i] > 0):
        figma['Assist1'].iloc[i] = 0
    if (figma['Assist2'].iloc[i] > 0) & (figma['Assist3'].iloc[i] > 0):
        figma['Assist2'].iloc[i] = 0

for i in range (0,len(figma)):
    if (figma['Cork/Start3'].iloc[i] > 0) & (figma['Cork/Start1'].iloc[i] > 0):
        figma['Cork/Start3'].iloc[i] = 0
    if (figma['Cork/Start1'].iloc[i] > 0) & (figma['Cork/Start2'].iloc[i] > 0):
        figma['Cork/Start1'].iloc[i] = 0
    if (figma['Cork/Start2'].iloc[i] > 0) & (figma['Cork/Start3'].iloc[i] > 0):
        figma['Cork/Start2'].iloc[i] = 0

In [51]:
# Same thing for Win, Assist, and Cork/Start

figma['Win1'] = figma['Win1'].fillna(0)
figma['Win2'] = figma['Win2'].fillna(0)
figma['Win3'] = figma['Win3'].fillna(0)
figma['Assist1'] = figma['Assist1'].fillna(0)
figma['Assist2'] = figma['Assist2'].fillna(0)
figma['Assist3'] = figma['Assist3'].fillna(0)
figma['Cork/Start1'] = figma['Cork/Start1'].fillna(0)
figma['Cork/Start2'] = figma['Cork/Start2'].fillna(0)
figma['Cork/Start3'] = figma['Cork/Start3'].fillna(0)

figma['Win'] = figma['Win1'] + figma['Win2'] + figma['Win3']
figma['Assist'] = figma['Assist1'] + figma['Assist2'] + figma['Assist3']
figma['Cork/Start'] = figma['Cork/Start1'] + figma['Cork/Start2'] + figma['Cork/Start3']

In [78]:
wcd['FiGMA_LWin%'] = 0.00
wcd['FiGMA_LAssist%'] = 0.00
wcd['FiGMA_LStart%'] = 0.00


for i in range(0,len(wcd)):
    wcd['FiGMA_LWin%'][i] = figma['Win'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]

for i in range(0,len(wcd)):
    wcd['FiGMA_LAssist%'][i] = figma['Assist'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]

for i in range(0,len(wcd)):
    wcd['FiGMA_LStart%'][i] = figma['Cork/Start'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]


<ipython-input-78-fcbc3948a902>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wcd['FiGMA_LWin%'][i] = figma['Win'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]
<ipython-input-78-fcbc3948a902>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wcd['FiGMA_LAssist%'][i] = figma['Assist'].loc[wcd['Team'][i],wcd['Set'][i],wcd['Game'][i],wcd['New_Date'][i]]
<ipython-input-78-fcbc3948a902>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a

In [79]:
wcd.to_csv('FiGMA_Aligned.csv')

# The moving averages should now be in the correct rows.

### Remove all FiGMA values that are not for the game being played in that leg. Then impute where 0 is the value for FiGMA (these are first games) use other team's average, then the league average for losers for that weekday. Additionally, get rid of FiGMA values where the 

In [2]:
wcd = pd.read_csv('FiGMA_Aligned.csv')

In [3]:
figs = wcd.copy()

In [4]:
figs['FiGMA_TDAC'] = figs['FiGMA_TDAC'].mask(figs['Cricket'] == False)
figs['FiGMA_TDA3'] = figs['FiGMA_TDA3'].mask(figs['301'] == False)
figs['FiGMA_TDA5'] = figs['FiGMA_TDA5'].mask(figs['Cricket'] | figs['301'])

In [5]:
figs['FiGMA_TDAC'] = figs['FiGMA_TDAC'].fillna(0.0)
figs['FiGMA_TDA3'] = figs['FiGMA_TDA3'].fillna(0.0)
figs['FiGMA_TDA5'] = figs['FiGMA_TDA5'].fillna(0.0)

In [6]:
# ID solo sets
masko = (figs['Set'] == 1) | (figs['Set'] == 3) | (figs['Set'] == 5) | (figs['Set'] == 8)

In [7]:
#Remove assist values for solo sets
figs['FiGMA_LAssist%'] = figs['FiGMA_LAssist%'].mask(masko)

In [16]:
# Take out all the Matches that may have the wrong date. 8395 legs in this case.
longseason = figs['Season_Day'] > 100
figs = figs.mask(longseason)

In [17]:
longseason.value_counts()

False    150435
Name: Season_Day, dtype: int64

In [23]:
# Take out any Win, Assist, Cork, for first games.
firstski = (figs['FiGMA_TDAC'] + figs['FiGMA_TDA3'] + figs['FiGMA_TDA5']) == 0.0

In [24]:
firstski.value_counts()

False    150435
dtype: int64

In [126]:
figs['Impute_AVG'] = firstski

In [127]:
avgs = figs.groupby(['Game','Tuesday'])['TDA'].mean()

In [138]:
figs['Impute_Avg'] = 0.0
for i in range(0,len(figs)):
    if figs['Impute_AVG'][i] == True:
        figs['Impute_Avg'][i] = float(avgs.loc[figs['Game'][i],figs['Tuesday'][i]])

<ipython-input-138-ef11e33fa4b6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  figs['Impute_Avg'][i] = float(avgs.loc[figs['Game'][i],figs['Tuesday'][i]])


In [27]:
figs

,Unnamed: 0,index,Unnamed: 0.1,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,...,Assist3,Cork/Start1,Cork/Start2,Cork/Start3,FiGMA_TDA5,FiGMA_TDA3,FiGMA_TDAC,FiGMA_LWin%,FiGMA_LAssist%,FiGMA_LStart%
0,0.0,0.0,0.0,Bacon,6-7-16,Home,1.1,501 SIDO,24.6,Jason Callan,...,NaN,1.0,NaN,NaN,29.935639,0.0,0.0,5.342948e-15,NaN,2.609024e-15
1,1.0,1.0,1.0,Motley Crew,6-7-16,Away,1.1,501 SIDO,24.4,Ken Molitoris,...,NaN,0.0,NaN,NaN,47.350000,0.0,0.0,3.750000e-01,NaN,3.750000e-01
2,2.0,2.0,2.0,Bacon,6-7-16,Home,1.2,501 SIDO,32.0,Jason Callan,...,NaN,0.0,NaN,NaN,29.935639,0.0,0.0,5.342948e-15,NaN,2.609024e-15
3,3.0,3.0,3.0,Motley Crew,6-7-16,Away,1.2,501 SIDO,30.8,Ken Molitoris,...,NaN,1.0,NaN,NaN,47.350000,0.0,0.0,3.750000e-01,NaN,3.750000e-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150430,150430.0,12725.0,69.0,Dark Nights,2-18-20,Away,9.2,501 SIDO,70.0,Jacob Zawa,...,0.0,NaN,NaN,0.0,52.307224,0.0,0.0,2.000000e-01,0.2,2.333333e-01
150431,150431.0,12726.0,70.0,Cork Busters,2-18-20,Home,9.3,501 SIDO,45.5,Tim Gilmore,...,0.0,NaN,NaN,0.0,54.534293,0.0,0.0,1.000000e-01,0.1,3.666667e-01
150432,150432.0,12727.0,71.0,Cork Busters,2-18-20,Home,9.3,501 SIDO,71.8,Eddie Palacios,...,0.0,NaN,NaN,0.0,54.534293,0.0,0.0,1.000000e-01,0.1,3.666667e-01
150433,150433.0,12728.0,72.0,Dark Nights,2-18-20,Away,9.3,501 SIDO,70.6,Karl Hamilton,...,0.0,NaN,NaN,1.0,52.307224,0.0,0.0,2.000000e-01,0.2,2.333333e-01


In [25]:
figs = figs.mask(firstski)

In [28]:
figs['Home'].value_counts()

1.0    68396
0.0    68252
Name: Home, dtype: int64

In [30]:
figs.to_csv("BigFigs2.csv")

In [31]:
#Split into home/away sets and then merge
homes = figs['Home/Away'] == 'Home'
aways = figs['Home/Away'] != 'Home'

In [32]:
homs = figs.mask(aways)
awas = figs.mask(homes)

In [33]:
homs.dropna(axis=0,how='all')

,Unnamed: 0,index,Unnamed: 0.1,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,...,Assist3,Cork/Start1,Cork/Start2,Cork/Start3,FiGMA_TDA5,FiGMA_TDA3,FiGMA_TDAC,FiGMA_LWin%,FiGMA_LAssist%,FiGMA_LStart%
0,0.0,0.0,0.0,Bacon,6-7-16,Home,1.1,501 SIDO,24.6,Jason Callan,...,NaN,1.0,NaN,NaN,29.935639,0.0,0.000000,5.342948e-15,NaN,2.609024e-15
2,2.0,2.0,2.0,Bacon,6-7-16,Home,1.2,501 SIDO,32.0,Jason Callan,...,NaN,0.0,NaN,NaN,29.935639,0.0,0.000000,5.342948e-15,NaN,2.609024e-15
52,52.0,52.0,52.0,Bacon,6-7-16,Home,8.1,Cricket,1.9,Ed Skoczylas,...,NaN,NaN,1.0,NaN,0.000000,0.0,1.745475,5.000000e-01,NaN,2.500000e-01
54,54.0,54.0,54.0,Bacon,6-7-16,Home,8.2,Cricket,1.4,Ed Skoczylas,...,NaN,NaN,0.0,NaN,0.000000,0.0,1.745475,5.000000e-01,NaN,2.500000e-01
56,56.0,56.0,56.0,Bacon,6-7-16,Home,8.3,Cricket,1.1,Ed Skoczylas,...,NaN,NaN,1.0,NaN,0.000000,0.0,1.745475,5.000000e-01,NaN,2.500000e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150424,150424.0,12719.0,63.0,Cork Busters,2-18-20,Home,9.1,501 SIDO,50.3,Eddie Palacios,...,0.0,NaN,NaN,0.0,54.534293,0.0,0.000000,1.000000e-01,0.1,3.666667e-01
150427,150427.0,12722.0,66.0,Cork Busters,2-18-20,Home,9.2,501 SIDO,57.3,Tim Gilmore,...,1.0,NaN,NaN,1.0,54.534293,0.0,0.000000,1.000000e-01,0.1,3.666667e-01
150428,150428.0,12723.0,67.0,Cork Busters,2-18-20,Home,9.2,501 SIDO,81.6,Eddie Palacios,...,0.0,NaN,NaN,0.0,54.534293,0.0,0.000000,1.000000e-01,0.1,3.666667e-01
150431,150431.0,12726.0,70.0,Cork Busters,2-18-20,Home,9.3,501 SIDO,45.5,Tim Gilmore,...,0.0,NaN,NaN,0.0,54.534293,0.0,0.000000,1.000000e-01,0.1,3.666667e-01


In [34]:
awas.dropna(axis=0,how='all')

,Unnamed: 0,index,Unnamed: 0.1,Team,Date,Home/Away,Leg_Code,Game,3DA,Player,...,Assist3,Cork/Start1,Cork/Start2,Cork/Start3,FiGMA_TDA5,FiGMA_TDA3,FiGMA_TDAC,FiGMA_LWin%,FiGMA_LAssist%,FiGMA_LStart%
1,1.0,1.0,1.0,Motley Crew,6-7-16,Away,1.1,501 SIDO,24.4,Ken Molitoris,...,NaN,0.0,NaN,NaN,47.350000,0.0,0.000000,0.375,NaN,0.375000
3,3.0,3.0,3.0,Motley Crew,6-7-16,Away,1.2,501 SIDO,30.8,Ken Molitoris,...,NaN,1.0,NaN,NaN,47.350000,0.0,0.000000,0.375,NaN,0.375000
53,53.0,53.0,53.0,Motley Crew,6-7-16,Away,8.1,Cricket,1.7,Kevin Askham,...,NaN,NaN,0.0,NaN,0.000000,0.0,2.656371,0.500,NaN,0.250000
55,55.0,55.0,55.0,Motley Crew,6-7-16,Away,8.2,Cricket,1.8,Kevin Askham,...,NaN,NaN,1.0,NaN,0.000000,0.0,2.656371,0.500,NaN,0.250000
57,57.0,57.0,57.0,Motley Crew,6-7-16,Away,8.3,Cricket,2.2,Kevin Askham,...,NaN,NaN,0.0,NaN,0.000000,0.0,2.656371,0.500,NaN,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150426,150426.0,12721.0,65.0,Dark Nights,2-18-20,Away,9.1,501 SIDO,30.8,Jacob Zawa,...,0.0,NaN,NaN,0.0,52.307224,0.0,0.000000,0.200,0.2,0.233333
150429,150429.0,12724.0,68.0,Dark Nights,2-18-20,Away,9.2,501 SIDO,64.8,Karl Hamilton,...,0.0,NaN,NaN,0.0,52.307224,0.0,0.000000,0.200,0.2,0.233333
150430,150430.0,12725.0,69.0,Dark Nights,2-18-20,Away,9.2,501 SIDO,70.0,Jacob Zawa,...,0.0,NaN,NaN,0.0,52.307224,0.0,0.000000,0.200,0.2,0.233333
150433,150433.0,12728.0,72.0,Dark Nights,2-18-20,Away,9.3,501 SIDO,70.6,Karl Hamilton,...,0.0,NaN,NaN,1.0,52.307224,0.0,0.000000,0.200,0.2,0.233333


In [35]:
#Looks like I need to collapse into sets first. Otherwise, there will be mismatches.
setcont = figs.groupby(['Match','Team','Set'])['FiGMA_TDAC','FiGMA_TDA3','FiGMA_TDA5','FiGMA_LWin%','FiGMA_LAssist%','FiGMA_LStart%','Home'].mean()

<ipython-input-35-9ccd46c561ae>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  setcont = figs.groupby(['Match','Team','Set'])['FiGMA_TDAC','FiGMA_TDA3','FiGMA_TDA5','FiGMA_LWin%','FiGMA_LAssist%','FiGMA_LStart%','Home'].mean()


In [36]:
setwin = figs.groupby(['Match','Team','Set'])['Win'].sum()

In [37]:
setwin.reset_index()
setwin.head(20)

Match                     Team            Set
5731394f0d39b85a663cde40  Dart Simpson    1.0    0.0
5732886467d7ed1e768bff66  Be Somebody!    1.0    2.0
                                          4.0    1.0
                          N.W.O.          1.0    0.0
                                          8.0    0.0
573288e067d7ed1e768bff84  Sparkle Motion  1.0    2.0
                                          8.0    1.0
5732895f67d7ed1e768bff98  One Too Many    1.0    1.0
                          Seems Legit     1.0    2.0
573289eb67d7ed1e768bffbc  Dart Team       1.0    0.0
                                          8.0    0.0
                          The Thatch Pub  1.0    2.0
                                          8.0    2.0
573a727b336cf5957426d104  Best Friends    8.0    2.0
                          FTW!            1.0    0.0
                                          2.0    0.0
                                          3.0    0.0
                                          4.0    1.0


In [38]:
setwin = setwin == 2

In [39]:
setwin.head(20)

Match                     Team            Set
5731394f0d39b85a663cde40  Dart Simpson    1.0    False
5732886467d7ed1e768bff66  Be Somebody!    1.0     True
                                          4.0    False
                          N.W.O.          1.0    False
                                          8.0    False
573288e067d7ed1e768bff84  Sparkle Motion  1.0     True
                                          8.0    False
5732895f67d7ed1e768bff98  One Too Many    1.0    False
                          Seems Legit     1.0     True
573289eb67d7ed1e768bffbc  Dart Team       1.0    False
                                          8.0    False
                          The Thatch Pub  1.0     True
                                          8.0     True
573a727b336cf5957426d104  Best Friends    8.0     True
                          FTW!            1.0    False
                                          2.0    False
                                          3.0    False
                   

In [40]:
setcont['Set_Win'] = setwin

In [41]:
setcont.head()

FiGMA_TDAC  FiGMA_TDA3  FiGMA_TDA5  \
Match                    Team         Set                                       
5731394f0d39b85a663cde40 Dart Simpson 1.0    0.000000         0.0   53.381677   
5732886467d7ed1e768bff66 Be Somebody! 1.0    0.000000         0.0   41.445080   
                                      4.0    2.833135         0.0    0.000000   
                         N.W.O.       1.0    0.000000         0.0   47.669012   
                                      8.0    2.820833         0.0    0.000000   

                                           FiGMA_LWin%  FiGMA_LAssist%  \
Match                    Team         Set                                
5731394f0d39b85a663cde40 Dart Simpson 1.0     0.500000             NaN   
5732886467d7ed1e768bff66 Be Somebody! 1.0     0.266667             NaN   
                                      4.0     0.333333    2.886580e-15   
                         N.W.O.       1.0     0.444444             NaN   
                                      8.0     0.333333             NaN   

                                           FiGMA_LStart%  Home  Set_Win  
Match                    Team         Set                                
5731394f0d39b85a663cde40 Dart Simpson 1.0       0.500000   1.0    False  
5732886467d7ed1e768bff66 Be Somebody! 1.0       0.200000   0.0     True  
                                      4.0       0.833333   0.0    False  
                         N.W.O.       1.0       0.222222   1.0    False  
                                      8.0       0.266667   1.0    False

In [42]:
others = setcont.mask(setcont['Home']==True)
sames = setcont.mask(setcont['Home']==False)

In [43]:
others = others.dropna(axis=0,how='all')
sames = sames.dropna(axis=0,how='all')

In [44]:
others = others.reset_index()

In [45]:
sames = sames.reset_index()

In [46]:
sames.head()

,Match,Team,Set,FiGMA_TDAC,FiGMA_TDA3,FiGMA_TDA5,FiGMA_LWin%,FiGMA_LAssist%,FiGMA_LStart%,Home,Set_Win
0,5731394f0d39b85a663cde40,Dart Simpson,1.0,0.000000,0.0,53.381677,0.500000,NaN,0.500000,1.0,0.0
1,5732886467d7ed1e768bff66,N.W.O.,1.0,0.000000,0.0,47.669012,0.444444,NaN,0.222222,1.0,0.0
2,5732886467d7ed1e768bff66,N.W.O.,8.0,2.820833,0.0,0.000000,0.333333,NaN,0.266667,1.0,0.0
3,5732895f67d7ed1e768bff98,Seems Legit,1.0,0.000000,0.0,34.561989,0.300000,NaN,0.183333,1.0,1.0
4,573289eb67d7ed1e768bffbc,Dart Team,1.0,0.000000,0.0,44.645983,0.200000,NaN,0.233333,1.0,0.0


In [47]:
others['Opp_Team'] = others['Team']
others['Opp_FiGMA_TDAC'] = others['FiGMA_TDAC']
others['Opp_FiGMA_TDA3'] = others['FiGMA_TDA3']
others['Opp_FiGMA_TDA5'] = others['FiGMA_TDA5']
others['Opp_FiGMA_LWin%'] = others['FiGMA_LWin%']
others['Opp_FiGMA_LAssist%'] = others['FiGMA_LAssist%']
others['Opp_FiGMA_LStart%'] = others['FiGMA_LStart%']

In [48]:
others = others(columns=["Match","Opp_Team","Set","Opp_FiGMA_TDAC","Opp_FiGMA_TDA3","Opp_FiGMA_TDA5","Opp_FiGMA_LWin%","Opp_FiGMA_LAssist%","Opp_FiGMA_LStart%"])

TypeError: 'DataFrame' object is not callable

In [49]:
others.head()

,Match,Team,Set,FiGMA_TDAC,FiGMA_TDA3,FiGMA_TDA5,FiGMA_LWin%,FiGMA_LAssist%,FiGMA_LStart%,Home,Set_Win,Opp_Team,Opp_FiGMA_TDAC,Opp_FiGMA_TDA3,Opp_FiGMA_TDA5,Opp_FiGMA_LWin%,Opp_FiGMA_LAssist%,Opp_FiGMA_LStart%
0,5732886467d7ed1e768bff66,Be Somebody!,1.0,0.000000,0.0,41.445080,0.266667,NaN,0.200000,0.0,1.0,Be Somebody!,0.000000,0.0,41.445080,0.266667,NaN,0.200000
1,5732886467d7ed1e768bff66,Be Somebody!,4.0,2.833135,0.0,0.000000,0.333333,2.886580e-15,0.833333,0.0,0.0,Be Somebody!,2.833135,0.0,0.000000,0.333333,2.886580e-15,0.833333
2,573288e067d7ed1e768bff84,Sparkle Motion,1.0,0.000000,0.0,43.984937,0.583333,NaN,0.416667,0.0,1.0,Sparkle Motion,0.000000,0.0,43.984937,0.583333,NaN,0.416667
3,573288e067d7ed1e768bff84,Sparkle Motion,8.0,2.145094,0.0,0.000000,0.500000,NaN,0.250000,0.0,0.0,Sparkle Motion,2.145094,0.0,0.000000,0.500000,NaN,0.250000
4,5732895f67d7ed1e768bff98,One Too Many,1.0,0.000000,0.0,41.559308,0.300000,NaN,0.300000,0.0,0.0,One Too Many,0.000000,0.0,41.559308,0.300000,NaN,0.300000


In [50]:
other = others[["Match","Opp_Team","Set","Opp_FiGMA_TDAC","Opp_FiGMA_TDA3","Opp_FiGMA_TDA5","Opp_FiGMA_LWin%","Opp_FiGMA_LAssist%","Opp_FiGMA_LStart%"]]

In [51]:
other.head()

,Match,Opp_Team,Set,Opp_FiGMA_TDAC,Opp_FiGMA_TDA3,Opp_FiGMA_TDA5,Opp_FiGMA_LWin%,Opp_FiGMA_LAssist%,Opp_FiGMA_LStart%
0,5732886467d7ed1e768bff66,Be Somebody!,1.0,0.000000,0.0,41.445080,0.266667,NaN,0.200000
1,5732886467d7ed1e768bff66,Be Somebody!,4.0,2.833135,0.0,0.000000,0.333333,2.886580e-15,0.833333
2,573288e067d7ed1e768bff84,Sparkle Motion,1.0,0.000000,0.0,43.984937,0.583333,NaN,0.416667
3,573288e067d7ed1e768bff84,Sparkle Motion,8.0,2.145094,0.0,0.000000,0.500000,NaN,0.250000
4,5732895f67d7ed1e768bff98,One Too Many,1.0,0.000000,0.0,41.559308,0.300000,NaN,0.300000


In [52]:
#Ugly Merge Time

beast = pd.merge(sames,other,on=['Match','Set'])

In [53]:
beast.to_csv("WCD_FiGMA_Matched24.csv")